<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Installing spark
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 33.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845498 sha256=12e5a9fe3191bf3f2c854303f5147543230445374a1ab1704898cec66727f46f
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/91/aa/66/700b503fd714b56462975ab7bf33485ec26677c3c990e67e9a
Successfully built pyspark


In [2]:
# Import libraries
import findspark
findspark.init()

import pandas as pd
from pyspark.sql.functions import desc
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegressionModel

In [3]:
# Start spark session

# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark Dataframes and SparkML Model") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

22/11/21 18:34:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Loading csv file from the url using pandas
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

searchterms = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv")

# Checking the first 5 rows of the dataframe
searchterms.head()

,day,month,year,searchterm
0,12,11,2021,mobile 6 inch
1,12,11,2021,mobile latest
2,12,11,2021,tablet wifi
3,12,11,2021,laptop 14 inch
4,12,11,2021,mobile 5g


In [5]:
# Load the pandas dataframe into a spark dataframe
sdf = spark.createDataFrame(searchterms)

In [6]:
# Checking the number of rows and columns in the spark dataframe
print((sdf.count(), len(sdf.columns)))

[Stage 0:=======>                                                   (1 + 7) / 8]

(10000, 4)


In [7]:
# Checking the first 5 rows of the spark dataframe
sdf.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [8]:
# Checking the datatypes of the each column in the dataframe
sdf.dtypes

[('day', 'bigint'),
 ('month', 'bigint'),
 ('year', 'bigint'),
 ('searchterm', 'string')]

In [9]:
# Checking the number of times the term `gaming laptop` is searched

sdf.filter(sdf['searchterm'] == 'gaming laptop').count()

499

In [10]:
# 5 most frequently used search terms
sdf.groupby('searchterm') \
.count() \
.sort(desc('count')) \
.show(5)

[Stage 6:===============================================>      (176 + 15) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [11]:
# Downloading and load a pretrained sales forecasting model from the url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

'''
The sales forecasting model is an already pretrained linear regression model. It infers that the input is year and and the output is sales.
Hence, the predictor variable is year and the response variable is sales. 
To understand how the model is built, please refer to the file `part-00000` in the metadata folder of the sales_prediction.model folder.
(Path: sales_prediction.model > metadata > part-0000.
'''

# Model download
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

# Extracting the content of the zip file
!tar -xvf model.tar.gz

--2022-11-21 18:34:27--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2022-11-21 18:34:28 (9.69 MB/s) - ‘model.tar.gz’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sal

In [12]:
# Loading the extracted contenet from the zip file (this will be the sales_prediction.model)

# Loading model
model = LinearRegressionModel.load('sales_prediction.model')

In [21]:
# Checking the intercept and coefficient of the linear regression model
print('The intercept of the model is', model.intercept)
print('The coefficient of the model is', model.coefficients[0])

The intercept of the model is -13019.989140447298
The coefficient of the model is 6.522567861288859


In [22]:
# This function predicts the sales for a chosen year
def predict(year):
    # A linear regression equation for prediction
    prediction = model.intercept + (model.coefficients[0] * year)
    # Returns the predicted value
    return prediction

In [27]:
# Prediction of sales for the year 2023
print(f'The sales prediction for 2023 is {predict(2023):.2f}')

The sales prediction for 2023 is 175.17
